# MOT lifetime
for making a vacuum pressure estimate

In [37]:
import scipy.io
import matplotlib.pyplot as plt
from numpy import *
import sys
sys.path.append('../')
from simfuncs import *

In [38]:
def process_csv(fname, newline='', cols=True, skiprows=1):
    rows = []
    with open(fname, 'r', newline=newline) as f:
        reader = csv.reader(f, delimiter=',')

        while skiprows > 0:
            reader.__next__()
            skiprows -= 1
        
        for row in reader:
            try:
                rows.append([float(x) for x in row if x != ''])
            except ValueError:
                print(f"row is problematic: \n {row}")

    if cols:
        return array(rows).T
    
    return rows

## Cs 3D MOT loading curves

In [23]:
# fpath 
files = [fpath+name for name in 
         []
        ]

In [24]:
# sample the file contents, get time increment
fname = files[0]
newline=''
with open(fname, 'r', newline=newline) as f:
    reader = csv.reader(f, delimiter=',')
    for i,row in enumerate(reader):
        if i == 1:
            t_incr = float(row[-1])     
        if i < 10:
            print(row)

['952.382620', ' -51.8245']
['2857.144492', ' -55.1114']
['4761.906365', ' -62.2933']
['6666.668237', ' -77.7545']
['8571.430110', ' -88.2259']
['10476.191982', ' -84.5513']
['12380.953854', ' -84.4927']
['14285.715727', ' -85.6648']
['16190.477599', ' -80.7151']
['18095.239471', ' -78.2957']


In [16]:
data = array([process_csv(f, newline='', cols=True, skiprows=0)
             for f in files])
data.shape

(5, 2, 262)

In [ ]:
gamma_b_per_P = 4.9e7 # cesium-non-cesium collisional losses [Torr^-1 s^-1]
beta = 4.013e7 # cesium-(hot)cesium collisional losses [Torr^-1 s^-1]

def loading_model(t,ns,tau):
    """
    fit MOT loading curve
    """    
    return ns*(1 - exp(-t/tau))

def gamma_model(tau,alpha_over_beta,gamma_b):
    """
    fit a Ns as a function of tau to get alpha/beta and 
    the background-induced loss rate gamma_b
    """
    return alpha_over_beta*(1 - gamma_b*tau)                

In [ ]:
## fluorescence curves w/ and w/o loading

ns_pts = []
tau_pts = []

for d in data:
    
    # fit the loading curve to get ns, tau
#     popt,popv = ...
    ns,tau = popt
    ns_pts.append()
    tau_pts.append()
    
# fit the ns,tau tuples to get alpha/beta and gamma_b
# popt,popv = ...
alpha_over_beta,gamma_b = popt

cs_pressure = ns_pts[0]/(alpha_over_beta*beta*tau_pts[0])
background_pressure = gamma_b/gamma_b_per_P

print("Cs background {cs_pressure} Torr")
print("non-Cs background {background_pressure} Torr")